In [ ]:
import pandas as pd 
import numpy as np 
from load_data import load_and_process_data, create_nutrient_constraints, load_all_menus, load_sample_file
from evaluation_function import calculate_harmony_matrix, get_top_n_harmony_pairs
from utils import diet_to_dataframe, count_menu_changes
import matplotlib.pyplot as plt
import seaborn as sns

from performance_metrics import PerformanceEvaluator
from nsga2_optimizer import NSGA2Optimizer
from nsga3_optimizer import NSGA3Optimizer
from spea2_optimizer import SPEA2Optimizer
from emoea_optimizer import EpsilonMOEAOptimizer

In [ ]:
# 데이터 로딩
def load_data(diet_db_path, menu_db_path, ingre_db_path):    
    diet_db = load_and_process_data(diet_db_path, menu_db_path, ingre_db_path)
    nutrient_constraints = create_nutrient_constraints()
    harmony_matrix, menus, menu_counts, _ = calculate_harmony_matrix(diet_db)
    all_menus = load_all_menus(menu_db_path, ingre_db_path)
    
    return diet_db, nutrient_constraints, harmony_matrix, menus, menu_counts, all_menus

# 현재 급식 제공 현황 분석
def analyze_current_menus(harmony_matrix, menus, menu_counts):
    print("=== 가장 많이 함께 나온 메뉴 조합 ===")
    top_5_pairs = get_top_n_harmony_pairs(harmony_matrix, menus, 5)
    for i, (menu1, menu2, frequency) in enumerate(top_5_pairs, 1):
        print(f"{i}. {menu1} - {menu2}: {frequency}회")
    
    print("\n=== 가장 자주 나온 메뉴 ===")
    top_5_menus = menu_counts.most_common(5)
    for i, (menu, occurrences) in enumerate(top_5_menus, 1):
        print(f"{i}. {menu}: {occurrences}회")

def compare_optimizers(initial_diet_path, diet_db, menu_db_path, ingre_db_path,
                    all_menus, nutrient_constraints, harmony_matrix,
                    generations: int = 100):
    weekly_diet = load_and_process_data(diet_db_path=initial_diet_path,
                                      menu_db_path=menu_db_path,
                                      ingre_db_path=ingre_db_path)
    
    # 최적화 알고리즘 초기화
    optimizers = {
        'NSGA-II': NSGA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'NSGA-III': NSGA3Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'SPEA2': SPEA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'ε-MOEA': EpsilonMOEAOptimizer(all_menus, nutrient_constraints, harmony_matrix)
    }

    # 초기 식단 분석
    initial_fitness = optimizers['NSGA-III'].fitness(diet_db, weekly_diet)
    #print_initial_analysis(weekly_diet, initial_fitness, nutrient_constraints)
    
    # 각 알고리즘별 최적화 수행
    all_results = {}
    for name, optimizer in optimizers.items():
        print(f"\n=== {name} 알고리즘 최적화 진행 중... ===")
        pareto_front = optimizer.optimize(diet_db, weekly_diet, generations)
        
        # 개선된 식단 선별
        improved_diets = []
        for optimized_diet in pareto_front:
            optimized_fitness = optimizer.fitness(diet_db, optimized_diet)
            improvements = calculate_improvements(initial_fitness, optimized_fitness)
            if sum(1 for imp in improvements if imp > 0) >= 3:
                improved_diets.append((optimized_diet, optimized_fitness, improvements))
        
        improved_diets = improved_diets[:5]
        all_results[name] = improved_diets
        
        # 결과 출력
        print_optimization_results(name, improved_diets, weekly_diet, nutrient_constraints)
    
    # 알고리즘 성능 비교
    evaluator = PerformanceEvaluator(diet_db, weekly_diet, optimizers)
    statistics = evaluator.run_comparison(generations, num_runs=10)
    print("\n=== 알고리즘 성능 비교 ===")
    
    return all_results

def print_initial_analysis(weekly_diet, initial_fitness, nutrient_constraints):
    print("=== 초기 식단 일일 평균 영양성분 ===")
    days = len(weekly_diet.meals) // 3
    for nutrient in nutrient_constraints.min_values.keys():
        total = sum(sum(menu.nutrients[nutrient] for menu in meal.menus) for meal in weekly_diet.meals)
        daily_avg = total / days
        min_val = nutrient_constraints.min_values[nutrient]
        max_val = nutrient_constraints.max_values[nutrient]
        print(f"{nutrient}: {daily_avg:.1f} (권장범위: {min_val}-{max_val})")
    
    total_cost = sum(sum(ingredient.price for menu in meal.menus 
                        for ingredient in menu.ingredients) 
                    for meal in weekly_diet.meals)
    print(f"\n총 식재료 비용: {total_cost:,.0f}원")

    print("=== 초기 식단 적합도 ===")
    print(f"영양: {initial_fitness[0]:.2f}")
    print(f"비용: {initial_fitness[1]:.2f}")
    print(f"조화: {initial_fitness[2]:.2f}")
    print(f"다양성: {initial_fitness[3]:.2f}")
    
    print("\n=== 초기 식단 ===")
    display(diet_to_dataframe(weekly_diet, "Initial Diet"))

def print_optimization_results(algorithm_name, improved_diets, initial_diet, nutrient_constraints):
    if improved_diets:
        print(f"\n=== {algorithm_name}: 개선된 식단 제안 ===")
        for i, (optimized_diet, optimized_fitness, improvements) in enumerate(improved_diets, 1):
            days = len(optimized_diet.meals) // 3
            print(f"\n[제안 식단 {i}]")
            #display(diet_to_dataframe(optimized_diet, f"{algorithm_name} - Optimized Diet {i}"))
            
            #print(f"\n[제안 식단 {i} 일일 평균 영양성분]")
            for nutrient in nutrient_constraints.min_values.keys():
                total = sum(sum(menu.nutrients[nutrient] for menu in meal.menus) for meal in optimized_diet.meals)
                daily_avg = total / days
                min_val = nutrient_constraints.min_values[nutrient]
                max_val = nutrient_constraints.max_values[nutrient]
                #print(f"{nutrient}: {daily_avg:.1f} (권장범위: {min_val}-{max_val})")
            
            optimized_cost = sum(sum(ingredient.price for menu in meal.menus for ingredient in menu.ingredients) for meal in optimized_diet.meals)
            #print(f"총 식재료 비용: {optimized_cost:,.0f}원")

            print("\n점수 개선율:")
            print(f"영양: {optimized_fitness[0]:.2f} ({improvements[0]:.2f}%)")
            print(f"비용: {optimized_fitness[1]:.2f} ({improvements[1]:.2f}%)")
            print(f"조화: {optimized_fitness[2]:.2f} ({improvements[2]:.2f}%)")
            print(f"다양성: {optimized_fitness[3]:.2f} ({improvements[3]:.2f}%)")
            
            menu_changes = count_menu_changes(initial_diet, optimized_diet)
            #print("\n카테고리별 메뉴 변경 비율:")
            for category, counts in menu_changes.items():
                percentage = (counts['changed'] / counts['total']) * 100 if counts['total'] > 0 else 0
                #print(f"{category}: {counts['changed']}/{counts['total']} ({percentage:.2f}%)")
    else:
        print(f"\n{algorithm_name}: 3가지 이상 개선된 식단을 찾지 못했습니다.")

def calculate_improvements(initial_fitness, optimized_fitness):
    improvements = []
    for init, opt in zip(initial_fitness, optimized_fitness):
        if init != 0:
            imp = (opt - init) / abs(init) * 100
        else:
            imp = float('inf') if opt > 0 else (0 if opt == 0 else float('-inf'))
        improvements.append(imp)
    return improvements

In [ ]:
# 데이터 로드
name = 'sarang'

diet_db_path = f'../data/sarang_DB/processed_DB/DIET_{name}.xlsx'
menu_db_path = f'../data/sarang_DB/processed_DB/Menu_ingredient_nutrient_{name}.xlsx'
ingre_db_path = f'../data/sarang_DB/processed_DB/Ingredient_Price_{name}.xlsx'

diet_db, nutrient_constraints, harmony_matrix, menus, menu_counts, all_menus = load_data(diet_db_path, menu_db_path, ingre_db_path)

# 현재 식단 현황 분석
#analyze_current_menus(harmony_matrix, menus, menu_counts)

# 초기 식단 파일 경로를 지정하여 최적화 실행
initial_diet_path = '../data/Weekly_diet_ex.xlsx'
results = compare_optimizers(
        initial_diet_path,
        diet_db, menu_db_path, ingre_db_path, all_menus, nutrient_constraints, harmony_matrix, 
        generations=100,
    )